In [1]:
import nltk
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline


# NLTK'nin Türkçe stop words'lerini indirelim



In [43]:

# Veri setini yükle ve temizle
df = pd.read_csv("IMDB Dataset.csv")

df3 = pd.read_csv("train.csv")
df4 = pd.read_csv("test.csv")


In [44]:
df2 = pd.read_csv("movie.csv")

In [53]:
df.head()

,metin,duygu
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [60]:
df2.head()

,metin,duygu
0,I grew up (b. 1965) watching and loving the Th...,negative
1,"When I put this movie in my DVD player, and sa...",negative
2,Why do people who do not know what a particula...,negative
3,Even though I have great interest in Biblical ...,negative
4,Im a die hard Dads Army fan and nothing will e...,positive


In [55]:
df3.head()

,metin,duygu
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [57]:
df4.head()

,metin,duygu
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [49]:
df = df.rename(columns={"review": "metin", "sentiment": "duygu"})
df2 = df2.rename(columns={"text": "metin", "label": "duygu"})
df3 = df3.rename(columns={"selected_text": "metin", "sentiment": "duygu"})
df4 = df4.rename(columns={"text": "metin", "sentiment": "duygu"})

In [54]:
df3.drop(columns=["textID","text"],inplace=True)

In [56]:
df4.drop(columns=["textID"],inplace=True)

In [58]:
df2['duygu'] = df2['duygu'].astype(int)

In [59]:

df2["duygu"] = df2["duygu"].apply(lambda x: "positive" if x == 1 else "negative")
df2.head(-1)

,metin,duygu
0,I grew up (b. 1965) watching and loving the Th...,negative
1,"When I put this movie in my DVD player, and sa...",negative
2,Why do people who do not know what a particula...,negative
3,Even though I have great interest in Biblical ...,negative
4,Im a die hard Dads Army fan and nothing will e...,positive
...,...,...
39994,My girlfriend has the habit of going to Blockb...,negative
39995,"""Western Union"" is something of a forgotten cl...",positive
39996,This movie is an incredible piece of work. It ...,positive
39997,My wife and I watched this movie because we pl...,negative


In [61]:
df4 = df4[df4['duygu'] != 'neutral'] 

In [62]:
df3 = df3[df3['duygu'] != 'neutral'] 

In [63]:

df3 = pd.concat([df, df2,df3,df4], ignore_index=True)
df3.head(-1)

,metin,duygu
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
108461,"..my mother just WON the City of Terrell, Texa...",positive
108462,"its at 3 am, im very tired but i can`t sleep ...",negative
108463,All alone in this old house again. Thanks for...,positive
108464,I know what you mean. My little dog is sinkin...,negative


In [64]:
nltk.download('stopwords')
from nltk.corpus import stopwords

# Türkçe stop words'leri alalım
stop_words = set(stopwords.words('English'))

# Veri temizleme fonksiyonu
def temizle_metin(metin):
    metin = re.sub(r'[^\w\s]', '', metin)  # Noktalama işaretlerini kaldır
    metin = metin.lower()
    metin = ' '.join([kelime for kelime in metin.split() if kelime not in stop_words])  # Stop words'leri kaldır
    return metin

df3['metin'] = df3['metin'].apply(temizle_metin)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Serkan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
df3.head(-1)

,metin,duygu
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production br br filming tech...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically theres family little boy jake thinks...,negative
4,petter matteis love time money visually stunni...,positive
...,...,...
108461,mother city terrell texas district 2 city coun...,positive
108462,3 im tired cant sleep try,negative
108463,alone old house thanks net keeps alive kicking...,positive
108464,know mean little dog sinking depression wants ...,negative


In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF modelini oluşturuyoruz

tfidf = TfidfVectorizer() 

X_tfidf = tfidf.fit_transform(df3["metin"])

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df3["duygu"], test_size=0.2, random_state=42)

In [68]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced')

model.fit(X_train, y_train)


LogisticRegression(class_weight='balanced')

In [69]:
model.score(X_test, y_test)

0.9100212040195446